In [2]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install streamlit_webrtc

  Obtaining dependency information for streamlit_webrtc from https://files.pythonhosted.org/packages/2e/18/d468e28786f847045238cd4d68bc0762b03014c5410e4828b918a10a34fa/streamlit_webrtc-0.47.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/992.6 kB ? eta -:--:--
     - -------------------------------------- 30.7/992.6 kB ? eta -:--:--
     --- ----------------------------------- 92.2/992.6 kB 1.3 MB/s eta 0:00:01
     ------- ------------------------------ 184.3/992.6 kB 1.4 MB/s eta 0:00:01
     ----------- -------------------------- 307.2/992.6 kB 1.9 MB/s eta 0:00:01
     ----------------- -------------------- 450.6/992.6 kB 2.2 MB/s eta 0:00:01
     ----------------------- -------------- 614.4/992.6 kB 2.3 MB/s eta 0:00:01
     -------------------------- ----------- 686.1/992.6 kB 2.3 MB/s eta 0:00:01
     ------------------------------ ------- 798.7/992.6 kB 2.3 MB/s eta 0:00:01
     ---------------------------------- --- 901.1/992.6 kB 2.3 MB/s eta 0:0

In [1]:
pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/b1/e9/5e59dec4830a9a0913cda2910f2c829065f447af7efc8338d6ff07c1f27c/mediapipe-0.10.3-cp311-cp311-win_amd64.whl.metadata
  Using cached mediapipe-0.10.3-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/05/33/5a6436146bda09c69decc456cfb54f41d52fbcf558fe91e6df7bdde6cce0/opencv_contrib_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl (199 kB)
Using cached mediapipe-0.10.3-cp311-cp311-win_amd64.whl (50.2 MB)
Using cached opencv_contrib_python-4.8.0.76-cp37-abi3-win_amd64.whl (44.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import streamlit as st
from streamlit_webrtc import webrtc_streamer
import av
import cv2 
import numpy as np 
import mediapipe as mp 
from keras.models import load_model
import webbrowser

model  = load_model("model.h5")
label = np.load("labels.npy")
holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

st.header("Emotion Based Music Recommender")

if "run" not in st.session_state:
	st.session_state["run"] = "true"

try:
	emotion = np.load("emotion.npy")[0]
except:
	emotion=""

if not(emotion):
	st.session_state["run"] = "true"
else:
	st.session_state["run"] = "false"

class EmotionProcessor:
	def recv(self, frame):
		frm = frame.to_ndarray(format="bgr24")

		##############################
		frm = cv2.flip(frm, 1)

		res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

		lst = []

		if res.face_landmarks:
			for i in res.face_landmarks.landmark:
				lst.append(i.x - res.face_landmarks.landmark[1].x)
				lst.append(i.y - res.face_landmarks.landmark[1].y)

			if res.left_hand_landmarks:
				for i in res.left_hand_landmarks.landmark:
					lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
					lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
			else:
				for i in range(42):
					lst.append(0.0)

			if res.right_hand_landmarks:
				for i in res.right_hand_landmarks.landmark:
					lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
					lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
			else:
				for i in range(42):
					lst.append(0.0)

			lst = np.array(lst).reshape(1,-1)

			pred = label[np.argmax(model.predict(lst))]

			print(pred)
			cv2.putText(frm, pred, (50,50),cv2.FONT_ITALIC, 1, (255,0,0),2)

			np.save("emotion.npy", np.array([pred]))

			
		drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_TESSELATION,
								landmark_drawing_spec=drawing.DrawingSpec(color=(0,0,255), thickness=-1, circle_radius=1),
								connection_drawing_spec=drawing.DrawingSpec(thickness=1))
		drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
		drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)


		##############################

		return av.VideoFrame.from_ndarray(frm, format="bgr24")

lang = st.text_input("Language")
singer = st.text_input("singer")

if lang and singer and st.session_state["run"] != "false":
	webrtc_streamer(key="key", desired_playing_state=True,
				video_processor_factory=EmotionProcessor)

btn = st.button("Recommend me songs")

if btn:
	if not(emotion):
		st.warning("Please let me capture your emotion first")
		st.session_state["run"] = "true"
	else:
		webbrowser.open(f"https://www.youtube.com/results?search_query={lang}+{emotion}+song+{singer}")
		np.save("emotion.npy", np.array([""]))
		st.session_state["run"] = "false"
